In [8]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time
import pandas as pd

# This is the path where I stored my chromedriver
# PATH = "/Users/mahi/Downloads/geckodriver"
PATH = "/usr/local/bin/chromedriver"

class AirbnbBot:
    # Class constructor that takes location, stay (Month, Week, Weekend)
    # Number of guests and type of guests (Adults, Children, Infants)
    def __init__(self, location, stay, number_guests, type_guests):
        self.location = location
        self.stay = stay
        self.number_guests = number_guests
        self.type_guests = type_guests
        self.driver = webdriver.Chrome()

    def search(self):
        # The driver will take us to the Airbnb website
        self.driver.get('https://www.airbnb.com')
        time.sleep(1)

        location = self.driver.find_element_by_xpath('//*[@id="bigsearch-query-location-input"]')
        
        try:
            # Use JavaScript to set the value of the location input directly
            self.driver.execute_script("arguments[0].value = '{}';".format(self.location), location)
            # Click on another element to trigger the search action
            search_button = self.driver.find_element_by_css_selector('button._sxfp92z')
            search_button.click()
        except StaleElementReferenceException:
            # If the element is stale, re-find the element before interacting with it
            location = self.driver.find_element_by_xpath('//*[@id="bigsearch-query-detached-query-input"]')
            self.driver.execute_script("arguments[0].value = '{}';".format(self.location), location)
            # Click on another element to trigger the search action
            search_button = self.driver.find_element_by_css_selector('button._sxfp92z')
            search_button.click()

        search_button = self.driver.find_element_by_css_selector('button._sxfp92z')
        search_button.click()

        # It was difficult to scrape every number on the calendar
        # so both the check-in and check-out dates are flexible.
        flexible = location.find_element_by_xpath('//*[@id="tab--tabs--1"]')
        flexible.click()

        if self.stay in ['Weekend', 'weekend']:
            weekend = self.driver.find_element_by_xpath('//*[@id="flexible_trip_lengths-weekend_trip"]/button')
            weekend.click()
            weekend.send_keys(Keys.RETURN)
        elif self.stay in ['Week', 'week']:
            week = self.driver.find_element_by_xpath('//*[@id="flexible_trip_lengths-one_week"]/button')
            week.click()
            week.send_keys(Keys.RETURN)
        elif self.stay in ['Month', 'month']:
            month = self.driver.find_element_by_xpath('//*[@id="flexible_trip_lengths-one_month"]/button')
            month.click()
            month.send_keys(Keys.RETURN)
        else:
            pass

        guest_button = self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div[1]/div/div/div[1]/div[1]/div/header/div/div[2]/div[2]/div/div/div/form/div[2]/div/div[5]/div[1]')
        guest_button.click()

        if self.type_guests in ['Adults', 'adults']:
            adults = self.driver.find_element_by_xpath('//*[@id="stepper-adults"]/button[2]')
            for num in range(int(self.number_guests)):
                adults.click()
        elif self.type_guests in ['Children', 'children']:
            children = self.driver.find_element_by_xpath('//*[@id="stepper-children"]/button[2]')
            for num in range(int(self.number_guests)):
                children.click()
        elif self.type_guests in ['Infants', 'infants']:
            infants = self.driver.find_element_by_xpath('//*[@id="stepper-infants"]/button[2]')
            for num in range(int(self.number_guests)):
                infants.click()
        else:
            pass

        x = self.driver.find_element_by_xpath('//*[@id="field-guide-toggle"]')
        x.click()
        x.send_keys(Keys.RETURN)

        search = self.driver.find_element_by_css_selector('button._sxfp92z')
        search.click()

    def scraping_airbnb(self):
        self.driver.maximize_window()
        src = self.driver.page_source
        soup = BeautifulSoup(src, features='lxml')
        list_of_houses = soup.find('div', class_="_fhph4u")

        type_of_property = list_of_houses.find_all('div', class_="_1tanv1h")
        list_type_property = [i.text for i in type_of_property]

        host_description = list_of_houses.find_all('div', class_='_5kaapu')
        list_host_description = [i.text for i in host_description]

        number_of_bedrooms_bathrooms = list_of_houses.find_all('div', class_="_3c0zz1")
        list_bedrooms_bathrooms = [i.text for i in number_of_bedrooms_bathrooms]
        bedrooms_bathrooms = list_bedrooms_bathrooms[::2]
        other_amenities = list_bedrooms_bathrooms[1::2]

        dates = list_of_houses.find_all('div', class_="_1v92qf0")
        list_dates = [date.text for date in dates]

        stars = list_of_houses.find_all('div', class_="_1hxyyw3")
        list_stars = [star.text[:3] for star in stars]

        prices = list_of_houses.find_all('div', class_="_1gi6jw3f")
        list_prices = [price.text for price in prices]

        airbnb_data = pd.DataFrame({
            'Type': list_type_property,
            'Host description': list_host_description,
            'Bedrooms & bathrooms': bedrooms_bathrooms,
            'Other amenities': other_amenities,
            'Date': list_dates,
            'Price': list_prices
        })

        airbnb_data.to_csv('Airbnb_data.csv', index=False)

if __name__ == '__main__':
    vacation = AirbnbBot('New York', 'week', '2', 'adults')
    vacation.search()
    time.sleep(2)
    vacation.scraping_airbnb()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button._sxfp92z"}
  (Session info: chrome=119.0.6045.199)
